In [18]:
!pip install streamlit pyngrok

In [33]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px

# Title
st.set_page_config(page_title="Superstore Dashboard", layout="wide")
st.title("Superstore Business Insights Dashboard")

# Load data
df = pd.read_csv("/content/Sample - Superstore.csv", encoding='latin1', parse_dates=['Order Date', 'Ship Date'])
df['Month'] = df['Order Date'].dt.to_period('M').astype(str)
df['Year'] = df['Order Date'].dt.year

# Sidebar filters
st.sidebar.header("Filters")

regions = st.sidebar.multiselect("Select Region", df['Region'].unique(), default=df['Region'].unique())
categories = st.sidebar.multiselect("Select Category", df['Category'].unique(), default=df['Category'].unique())
years = st.sidebar.slider("Select Year Range", int(df['Year'].min()), int(df['Year'].max()), (int(df['Year'].min()), int(df['Year'].max())))

# Filtered data
filtered_df = df[
    (df['Region'].isin(regions)) &
    (df['Category'].isin(categories)) &
    (df['Year'].between(*years))
]

# KPIs
total_sales = filtered_df['Sales'].sum()
total_profit = filtered_df['Profit'].sum()
average_order_value = filtered_df['Sales'].sum() / filtered_df['Quantity'].sum()

col1, col2, col3 = st.columns(3)
col1.metric("Total Sales", f"${total_sales:,.2f}")
col2.metric("Total Profit", f"${total_profit:,.2f}")
col3.metric("Avg Order Value", f"${average_order_value:,.2f}")

st.markdown("---")

# Sales Over Time
sales_trend = filtered_df.groupby('Month')['Sales'].sum().reset_index()
fig1 = px.line(sales_trend, x='Month', y='Sales', title='Sales Over Time', markers=True)
st.plotly_chart(fig1, use_container_width=True)

# Sales by Region
sales_region = filtered_df.groupby('Region')['Sales'].sum().reset_index()
fig2 = px.bar(sales_region, x='Region', y='Sales', title='Sales by Region', color='Region', text_auto='.2s')
st.plotly_chart(fig2, use_container_width=True)

# Sales by Category
sales_category = filtered_df.groupby('Category')['Sales'].sum().reset_index()
fig3 = px.pie(sales_category, values='Sales', names='Category', title='Sales by Category')
st.plotly_chart(fig3, use_container_width=True)

# Correlation Heatmap
correlation = filtered_df[['Sales', 'Profit', 'Discount', 'Quantity']].corr()
fig4 = px.imshow(correlation, text_auto=True, title="Correlation Between Sales Metrics")
st.plotly_chart(fig4, use_container_width=True)

# Summary
st.markdown("## Summary of Key Insights")
top_state = filtered_df.groupby('State')['Sales'].sum().sort_values(ascending=False).idxmax()
top_category = filtered_df.groupby('Category')['Sales'].sum().sort_values(ascending=False).idxmax()

st.markdown(f"""
- **Top Performing State:** `{top_state}`
- **Most Popular Category:** `{top_category}`
- **Sales Growth Trend Available by Month Above**
""")


Overwriting app.py


In [31]:
from pyngrok import ngrok
!ngrok authtoken 2iVirvlJyLSNLLptyt7HZuigzBJ_68MmdVdXo5mindnLDG6Ak

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [35]:
from pyngrok import ngrok

# Kill existing tunnels
ngrok.kill()

# Run Streamlit app
!streamlit run app.py &>/content/logs.txt &

# Expose via public URL
public_url = ngrok.connect("http://localhost:8501")
print(f"🌐 Your Streamlit app is live at: {public_url}")


🌐 Your Streamlit app is live at: NgrokTunnel: "https://62ff-35-237-89-199.ngrok-free.app" -> "http://localhost:8501"
